In [44]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)

import backtrader as bt
import backtrader.indicators as btind
import yfinance as yf
import pandas as pd

# --- MACD 策略 ---
class MACDCrossoverFull(bt.Strategy):
    params = (('fast_period', 12), ('slow_period', 26), ('signal_period', 9))

    def __init__(self):
        self.pending_buy = False     # 昨天決定今天買
        self.pending_sell = False    # 昨天決定今天賣
        self.order = None

        self.macd = btind.MACD(period_me1=self.p.fast_period,
                               period_me2=self.p.slow_period,
                               period_signal=self.p.signal_period)
        self.crossover = btind.CrossOver(self.macd.macd, self.macd.signal)

    def notify_order(self, order):
        # Submitted / Accepted → 代表掛單成功，但尚未成交
        if order.status in [order.Submitted, order.Accepted]:
            return

        if order.status == order.Completed:
            action = "買入" if order.isbuy() else "賣出"
            print(f"{self.data.datetime.date(0)} 📌 {action}成交！"
                  f" 成交價: {order.executed.price:.2f}, "
                  f"股數: {order.executed.size}, "
                  f"現金剩餘: {self.broker.getcash():.2f}")
            self.order = None  # 解鎖

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            print(f"{self.data.datetime.date(0)} ⚠️ 訂單失敗（{order.getstatusname()}），已解鎖")
            self.order = None  # 失敗也要解鎖！

    def notify_trade(self, trade):
        if trade.isclosed:
            print(f"{self.data.datetime.date(0)} 🤝 交易平倉：毛利 {trade.pnl:.2f}, 淨利 {trade.pnlcomm:.2f}")

    def next(self):
        # 1️⃣ 若還有訂單等待成交 → 不做事
        if self.order:
            return

        # 2️⃣ 先執行前一天決定的交易 → 用今天開盤價成交
        # 賣在先（避免同一天又買又賣）
        if self.pending_sell and self.position:
            size = self.position.size
            self.order = self.sell(size=size)
            print(f"{self.data.datetime.date(0)} ⛔️ 全部賣出（執行昨日死叉）"
                  f" 開盤價: {self.data.open[0]:.2f}, 股數: {size}")
            self.pending_sell = False
            return  # 今日已執行，退出

        # 再處理買
        if self.pending_buy and not self.position:
            cash = self.broker.getcash()
            open_price = self.data.open[0]
            size = int(cash // open_price)

            if size > 0:
                self.order = self.buy(size=size)
                print(f"{self.data.datetime.date(0)} 🚀 全倉買進（執行昨日金叉）"
                      f" 開盤價: {open_price:.2f}, 股數: {size}")
            else:
                print(f"{self.data.datetime.date(0)} ⚠️ 現金不足無法買進")

            self.pending_buy = False
            return

        # 3️⃣ 使用今日收盤資料判斷明日行動
        if self.crossover[0] > 0:
            self.pending_buy = True     # 金叉 → 明天買
            self.pending_sell = False
            # print(f"{self.data.datetime.date(0)} ▶️ 金叉出現，預約明天買")
        elif self.crossover[0] < 0:
            self.pending_sell = True    # 死叉 → 明天賣
            self.pending_buy = False
            # print(f"{self.data.datetime.date(0)} ◀️ 死叉出現，預約明天賣")



# --- Buy & Hold 策略 (與 Sizer 配合) ---
class BuyAndHoldFull(bt.Strategy):
    def __init__(self):
        self.buy_signal = False  # 當天觸發購買信號
        self.order = None

    def next(self):
        # 如果還沒買，觸發購買信號
        if not self.position and not self.buy_signal:
            self.buy_signal = True
            return

        # 前一天觸發 buy_signal → 今天開盤執行
        if self.buy_signal and not self.position:
            open_price = self.data.open[0]
            cash = self.broker.getcash()

            size = int(cash // open_price)

            if size > 0:
                self.order = self.buy(size=size)
                print(f"{self.data.datetime.date(0)} 🚀 全倉買進！價格: {open_price:.2f}, 股數: {size}")
            else:
                print(f"{self.data.datetime.date(0)} ⚠️ 現金不足，無法買入")

            self.buy_signal = False

    def notify_order(self, order):
        if order.status in [order.Completed]:
            if order.isbuy():
                print(f"{self.data.datetime.date(0)} 📌 成交！成交價: {order.executed.price:.2f}, "
                      f"股數: {order.executed.size}")



# --- 下載資料 (加上 auto_adjust=True 確保使用調整後價格) ---
STOCK_TICKER = '2330.TW'
BENCH_TICKER = '2330.TW'
START = '2015-01-01'
END = '2024-12-30'

# **重要修正：加上 auto_adjust=True 確保使用調整後價格**
df2330 = yf.download(STOCK_TICKER, start=START, end=END, auto_adjust=True)
df0050 = yf.download(BENCH_TICKER, start=START, end=END, auto_adjust=True)

# 如果 auto_adjust=True，通常不會有多級索引，但保留程式碼以防萬一
if isinstance(df2330.columns, pd.MultiIndex):
    df2330.columns = df2330.columns.droplevel(1)

if isinstance(df0050.columns, pd.MultiIndex):
    df0050.columns = df0050.columns.droplevel(1)


# --- 回測 MACD 策略 ---
data1 = bt.feeds.PandasData(dataname=df2330)
cerebro1 = bt.Cerebro()
cerebro1.addstrategy(MACDCrossoverFull)
cerebro1.adddata(data1)
cerebro1.broker.setcash(100000)
print("\n🚀 回測 MACD 策略中 ...")
cerebro1.run()
macd_final_value = cerebro1.broker.getvalue()
print("===========================")
# --- 回測 Buy & Hold 策略 ---
data2 = bt.feeds.PandasData(dataname=df0050)
cerebro2 = bt.Cerebro()
cerebro2.addsizer(bt.sizers.FixedSize, stake=100)
cerebro2.addstrategy(BuyAndHoldFull)
cerebro2.adddata(data2)
cerebro2.broker.setcash(100000)
print("\n📌 回測 Buy & Hold 策略中 ...")
cerebro2.run()
bh_final_value = cerebro2.broker.getvalue()


# --- 結果輸出 ---
print("\n===========================")
print("📊 回測結果比較")
print(f"期間: {START} 至 {END}")
print("---------------------------")
print(f"MACD ({STOCK_TICKER}) 最終資金: {macd_final_value:.2f}  ─ 報酬率: {(macd_final_value/100000-1):.2%}")
print(f"B&H ({BENCH_TICKER}) 最終資金: {bh_final_value:.2f} ─ 報酬率: {(bh_final_value/100000-1):.2%}")

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed



🚀 回測 MACD 策略中 ...
2015-03-20 🚀 全倉買進（執行昨日金叉） 開盤價: 111.83, 股數: 894
2015-03-23 ⚠️ 訂單失敗（Margin），已解鎖
2015-04-27 🚀 全倉買進（執行昨日金叉） 開盤價: 112.94, 股數: 885
2015-04-28 ⚠️ 訂單失敗（Margin），已解鎖
2015-05-12 🚀 全倉買進（執行昨日金叉） 開盤價: 108.51, 股數: 921
2015-05-13 ⚠️ 訂單失敗（Margin），已解鎖
2015-06-16 🚀 全倉買進（執行昨日金叉） 開盤價: 105.56, 股數: 947
2015-06-17 📌 買入成交！ 成交價: 105.56, 股數: 947, 現金剩餘: 38.82
2015-07-10 ⛔️ 全部賣出（執行昨日死叉） 開盤價: 103.96, 股數: 947
2015-07-13 📌 賣出成交！ 成交價: 104.34, 股數: -947, 現金剩餘: 98851.33
2015-07-13 🤝 交易平倉：毛利 -1148.67, 淨利 -1148.67
2015-07-22 🚀 全倉買進（執行昨日金叉） 開盤價: 105.87, 股數: 933
2015-07-23 📌 買入成交！ 成交價: 104.34, 股數: 933, 現金剩餘: 1499.63
2015-07-24 ⛔️ 全部賣出（執行昨日死叉） 開盤價: 105.10, 股數: 933
2015-07-27 📌 賣出成交！ 成交價: 102.82, 股數: -933, 現金剩餘: 97430.15
2015-07-27 🤝 交易平倉：毛利 -1421.18, 淨利 -1421.18
2015-08-03 🚀 全倉買進（執行昨日金叉） 開盤價: 105.49, 股數: 923
2015-08-04 📌 買入成交！ 成交價: 102.82, 股數: 923, 現金剩餘: 2527.81
2015-10-29 ⛔️ 全部賣出（執行昨日死叉） 開盤價: 105.87, 股數: 923
2015-10-30 📌 賣出成交！ 成交價: 104.34, 股數: -923, 現金剩餘: 98836.11
2015-10-30 🤝 交易平倉：毛利 1405.96, 淨利 1405.96
2